In [1]:
from pyspark import SparkContext, SparkConf
import sys
import os
import random as rand
import CountTriangles as ct
from collections import defaultdict

In [2]:
conf = SparkConf().setAppName('TriangleCountExercise')
sc = SparkContext(conf=conf)


23/03/25 15:40:45 WARN Utils: Your hostname, MacBook-Pro-di-Nicola.local resolves to a loopback address: 127.0.0.1; using 10.0.8.93 instead (on interface en0)
23/03/25 15:40:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/25 15:40:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# 1. Read number of partitions
C = 3
data_path = "facebook_small.txt"
assert os.path.isfile(data_path), "File or folder not found"
docs = sc.textFile(data_path,minPartitions=C).cache()
docs.repartition(numPartitions=C)

MapPartitionsRDD[6] at coalesce at DirectMethodHandleAccessor.java:104

In [4]:
def CountTriangles(edges):
    print(edges)
    # Create a defaultdict to store the neighbors of each vertex
    neighbors = defaultdict(set)
    for edge in edges:
        u, v = edge
        neighbors[u].add(v)
        neighbors[v].add(u)

    # Initialize the triangle count to zero
    triangle_count = 0

    # Iterate over each vertex in the graph.
    # To avoid duplicates, we count a triangle <u, v, w> only if u<v<w
    for u in neighbors:
        # Iterate over each pair of neighbors of u
        for v in neighbors[u]:
            if v > u:
                for w in neighbors[v]:
                    # If w is also a neighbor of u, then we have a triangle
                    if w > v and w in neighbors[u]:
                        triangle_count += 1
    # Return the total number of triangles in the graph
    return triangle_count


In [5]:
def assign_random_key(docs, C):
    edge_list = []
    for edge in docs.split(','):
        edge_list.append(edge)
    return [(rand.randint(0,C-1), edge_list)]

In [9]:
def count_triangles_task_two(docs, C):
    tri_count = (docs.flatMapValues(lambda x: assign_random_key(x, C)) # <-- MAP PHASE (R1)
                .groupByKey()                                    # <-- SHUFFLE+GROUPING
                 
                .reduceByKey(CountTriangles)                     # <-- REDUCE PHASE (R1)
                .reduceByKey(lambda x, y : x + y)
                )
    return tri_count

In [10]:
# 2-ROUND WORD COUNT - RANDOM KEYS ASSIGNED IN MAP PHASE
print("Number of triangles task two =", count_triangles_task_two(docs, C).count())

Number of triangles task two = 10
